In [1]:
###################################################################################################
###                             Motif Enrichment                                ###
###################################################################################################

#load libraries
library(tidyverse)
library(devtools)
library(motifmatchr)
library(BiocParallel)
load_all("/home/jpm73279/genome_downloads/BS_genomes/BSgenome.Sb")
library(Matrix)
library(GenomicAlignments)
library(dplyr)
library(universalmotif)  # manipulating motif representations



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.1     
✔ ggplot2   3.4.4          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis



ℹ Loading BSgenome.Sb
Loading required package: BSgenome

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union




In [2]:
### Load Motifs for Analysis
core_motifs <- read_meme("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/00.data/conserved_consensus_motifs_Fig2B.meme")
de_novo_motifs <- read_meme("/home/jpm73279/genome_downloads/C4_markers/found_motifs/combined_unique_motifs.meme")
ms_de_novo_motifs <- read_meme("/home/jpm73279/genome_downloads/C4_markers/found_motifs/Ms.specific_CGTCGT_motif.comparative.meme")

all_assy_motifs <- c(core_motifs, de_novo_motifs, ms_de_novo_motifs)




all_assy_motifs_converted <- convert_motifs(all_assy_motifs, class = "TFBSTools-PWMatrix")

update_name <- function(pwm) {
    if (!is.null(pwm@name)) {
        pwm@ID <- pwm@name
    }
    return(pwm)
}

library(TFBSTools)
pw_matrix_list_converted <- lapply(all_assy_motifs_converted, update_name)
pw_matrix_list_converted <- lapply(pw_matrix_list_converted, PWMatrixList)
pw_matrix_list_converted <- do.call(c, pw_matrix_list_converted)


Attaching package: ‘TFBSTools’


The following object is masked from ‘package:Matrix’:

    Matrix




In [4]:
marker_file_paths <- c("/home/jpm73279/genome_downloads/C4_markers/Sb.c4_markers.bed")

# Function to read and process the file
process_file <- function(file_path) {
  species <- substr(basename(file_path), 1, 2)
  read_delim(file_path, delim = "\t", col_names = c("chrom", "start", "end", "geneID", "name", "type")) %>% 
    dplyr::select("chrom","start","end","geneID","name","type") %>%
    dplyr::mutate(species = (species))
}

# Apply the function to each file and store results in a list
list_of_dataframes <- lapply(marker_file_paths, process_file)

# Optionally combine all dataframes into one if needed
markers <- bind_rows(list_of_dataframes) %>% 
    dplyr::select(geneID,name,type,species)

Rows: 33 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
process_acr_data <- function(file_path, species_abbr, cell_type_filter, markers, genome) {
    # Read and annotate the data
    annotated_data <- readr::read_delim(file_path, delim = "\t", col_names = FALSE, col_types = "cccccccccc") %>%
        dplyr::mutate(species = species_abbr)

    # Combine and process data
  combined_acrs_species <- annotated_data %>%
        dplyr::ungroup() %>%
         dplyr::left_join(., markers, by = c("species", "X5" = "name"))  %>%
        dplyr::rename(name = X5) %>% 
        dplyr::rename("enzyme_location" = type, 
                      "locus" = name, 
                      "acr_name_type" = X9) %>%
        tidyr::separate(acr_name_type, into = c("acr", "acr_cell_type_specific_class"), sep = ";") %>%
        dplyr::mutate(acr_cell_type_specific_class = dplyr::case_when(
            acr_cell_type_specific_class == "bundle_sheath,procambial_meristem" ~ "bundle_sheath",
            acr_cell_type_specific_class == "bundle_sheath,procambium" ~ "bundle_sheath", 
            acr_cell_type_specific_class == "mesophyll,bundle_sheath" ~ "mesophyll_bundle_sheath",
            TRUE ~ acr_cell_type_specific_class)) %>%
        tidyr::separate(locus, into = c("species_other", "loci"), sep = "__", remove = FALSE) %>%
        tidyr::separate(loci, into = c("gene_family", "number"), sep = "_", remove = FALSE)

    # Duplicate rows with "mesophyll,bundle_sheath" and assign individual classes
    mesophyll <- combined_acrs_species %>%
        dplyr::filter(acr_cell_type_specific_class == "mesophyll_bundle_sheath") %>%
        dplyr::mutate(acr_cell_type_specific_class = "mesophyll")
    
    bundle_sheath <- combined_acrs_species %>%
        dplyr::filter(acr_cell_type_specific_class == "mesophyll_bundle_sheath") %>%
        dplyr::mutate(acr_cell_type_specific_class = "bundle_sheath")


    combined_acrs_species <- combined_acrs_species %>%
        dplyr::filter(acr_cell_type_specific_class != "mesophyll_bundle_sheath") %>%
        dplyr::bind_rows(mesophyll,bundle_sheath) %>%
        dplyr::filter(!is.na(locus)) %>%
        dplyr::filter(acr_cell_type_specific_class %in% cell_type_filter) %>% 
        dplyr::rename(chr = X6,
                 start = X7,
                 end = X8,
                 acr_id = acr,
                      score = acr_cell_type_specific_class)

    # Generate GRanges object
    granges_data <-  GRanges(combined_acrs_species, seqinfo=seqinfo(genome))


    return(granges_data)
}



# file_path <- "/scratch/jpm73279/comparative_single_cell/07.call.ACRs/10.C4_gene_relationship_only_single_gene/all_acr_intersections/zm.extended_c4_genes.acr_intersection.all.bed"  # Replace with your file path
# species_abbr <- "Zm"                   # Replace with your species abbreviation
# cell_type_filter <- c("broadly_accessible")  # Replace with your cell type filter

# zm_acrs_cts_gr <- process_acr_data(file_path, species_abbr, cell_type_filter, markers)


In [14]:
process_and_convert_conservation_data <- function(file_path, species, filter_acr_ids, genome) {
  # Read and process the file
  conservation_data <- readr::read_delim(file_path, 
             col_names = c("qChr", "qStart", "qEnd", "acr_name", "pval", "refFile", "rChr", "rStart", "rEnd", "rName", "rWidth", "strand"),
             col_types = "cccccccccccc") %>%
    dplyr::mutate(species = species) %>%
    tidyr::separate(acr_name, into = c("acr", "acr_cell_type_specific_class", "locus"), sep = ";") %>%
    dplyr::mutate(
      refFile = stringr::str_replace(refFile, "\\.all_combined_regions\\.passing_regions\\.blast_passing_regions\\.intersecting_regions\\.ref\\.frac\\.bed", ""),
      refFile = stringr::str_replace(refFile, ".*\\.vs\\.", "")
    ) %>%
    dplyr::filter(rName != ".") %>%
    tidyr::separate(rName, into = c("RefFrom", "racr_name", "rcell_type", "gene_family", "gene_family2"), sep = ";") %>%
    dplyr::select(acr, acr_cell_type_specific_class, locus, racr_name, rcell_type, gene_family, rWidth, species, rChr, rStart, rEnd) %>%
    tidyr::separate(racr_name, into = c("rspecies", "racr"), sep = "_(.*)", extra = "merge", remove = FALSE) %>%
    dplyr::mutate(acr_cell_type_specific_class = case_when(acr_cell_type_specific_class == "bundle_sheath,procambial_meristem" ~ "bundle_sheath",
                                                          acr_cell_type_specific_class == "bundle_sheath,procambium" ~ "bundle_sheath", 
                                                          TRUE ~ acr_cell_type_specific_class)) %>%
    dplyr::mutate(qacr = stringr::str_c(species, acr, sep = "_"))

  # Convert to GRanges
    convert_to_conservation_granges <- conservation_data %>% 
        dplyr::rename(chr = rChr,
                 start = rStart,
                 end = rEnd,
                 acr_id = acr,
                score = rWidth) %>% 
    #filter for only the ACRs which are CTS from the above analysis
        dplyr::filter(acr_id %in% filter_acr_ids)

    conservation_granges <- GRanges(convert_to_conservation_granges, seqinfo=seqinfo(genome))


  return(conservation_granges)
}

# conserved_regions_granges <- process_and_convert_conservation_data("/home/jpm73279/genome_downloads/C4_markers/conservation/Zm.all.ACR_conservation.intersection.bed",
#                                      "Zm",
#                                       filter_acrs_cell_type_specifc$acr,
#                                     BSgenome.Zm_B73)

In [7]:
# process_and_merge_granges <- function(conserved_gr, reference_gr) {
#     # Combine and reduce conserved GRanges
#     conserved_gr_merrged <- GenomicRanges::reduce(conserved_gr)

#     # Split conserved GRanges by acr_id and reduce each subset
#     split_granges <- split(conserved_gr, conserved_gr$acr_id)
#     reduced_granges_list <- lapply(split_granges, GenomicRanges::reduce)

#     # Reinitialize conserved_gr_merrged
#     conserved_gr_merrged <- GRanges()

#     # Loop over the list and assign names as metadata
#     for (name in names(reduced_granges_list)) {
#         gr <- reduced_granges_list[[name]]
#         mcols(gr)$acr_ID <- name
#         conserved_gr_merrged <- c(conserved_gr_merrged, gr)
#     }

#     # Annotate the conserved regions
#     overlaps <- GenomicRanges::findOverlaps(conserved_gr_merrged, reference_gr)
#     mcols(conserved_gr_merrged) <- mcols(reference_gr)[subjectHits(overlaps), ]
#     conserved_gr_merrged$conserved_regions <- "conserved"

#     # Annotate the Non-Conserved Regulatory Regions
#     results <- GenomicRanges::setdiff(reference_gr, conserved_gr_merrged, ignore.strand = TRUE)
#     revmap <- GenomicRanges::findOverlaps(results, reference_gr, select="arbitrary")
#     mcols(results) <- mcols(reference_gr)[revmap, , drop=FALSE]
#     results$conserved_regions <- "non_conserved"

#     # Merge conserved and non-conserved regions
#     merged_regions <- c(results, conserved_gr_merrged)

#     return(merged_regions)
# }



In [92]:
process_and_merge_granges <- function(conserved_gr, reference_gr) {
    # Combine and reduce conserved GRanges
    conserved_gr_merrged <- GenomicRanges::reduce(conserved_gr)

    # Split conserved GRanges by acr_id and reduce each subset
    split_granges <- split(conserved_gr, conserved_gr$acr_id)
    reduced_granges_list <- lapply(split_granges, GenomicRanges::reduce)

    # Reinitialize conserved_gr_merrged
    conserved_gr_merrged <- GRanges()

    # Loop over the list and assign names as metadata
    for (name in names(reduced_granges_list)) {
        gr <- reduced_granges_list[[name]]
        mcols(gr)$acr_ID <- name
        conserved_gr_merrged <- c(conserved_gr_merrged, gr)
    }

    # Annotate the conserved regions
    overlaps <- GenomicRanges::findOverlaps(conserved_gr_merrged, reference_gr)

    # Extract overlapping indices
    overlapping_indices <- subjectHits(overlaps)

    # Ensure length match
    if (length(overlapping_indices) == length(conserved_gr_merrged)) {
        # Transfer metadata columns
        mcols(conserved_gr_merrged) <- mcols(reference_gr)[overlapping_indices, ]
        conserved_gr_merrged$conserved_regions <- "conserved"
    } else {
        # Subset conserved_gr_merrged to include only overlapping ranges
        overlapping_indices_conserved <- queryHits(overlaps)
        conserved_gr_merrged_subset <- conserved_gr_merrged[overlapping_indices_conserved]

        # Transfer metadata columns to the subset
        mcols(conserved_gr_merrged_subset) <- mcols(reference_gr)[overlapping_indices, ]
        conserved_gr_merrged_subset$conserved_regions <- "conserved"

        # Optionally, handle the non-overlapping regions of conserved_gr_merrged
        # For example, you might want to mark them as 'non_conserved' or process them differently

        # Replace the original conserved_gr_merrged with the subset
        conserved_gr_merrged <- conserved_gr_merrged_subset
}

    # Annotate the Non-Conserved Regulatory Regions
    results <- GenomicRanges::setdiff(reference_gr, conserved_gr_merrged, ignore.strand = TRUE)
    revmap <- GenomicRanges::findOverlaps(results, reference_gr, select="arbitrary")
    mcols(results) <- mcols(reference_gr)[revmap, , drop=FALSE]
    results$conserved_regions <- "non_conserved"

    # Merge conserved and non-conserved regions
    merged_regions <- c(results, conserved_gr_merrged)

    return(merged_regions)
}



In [104]:
match_and_annotate_motifs <- function(pwm_matrix_list, merged_regions, genome, p_cutoff) {
    
    extractIDDataFrame <- function(pwMatrixList) {
          ids <- sapply(pwMatrixList@listData, function(item) item@ID)
          ids <- gsub("chr", "", ids, fixed = TRUE)
          indices <- seq_along(ids)
  
          data.frame(group = indices, TF_ID = ids)
    }
    motif_id_index <- extractIDDataFrame(pwm_matrix_list)

                
    # Matching motifs
    matches_grl <- matchMotifs(
        pwm_matrix_list, 
        merged_regions, 
        genome = genome, 
        p.cutoff = p_cutoff, 
        out = "positions"
    )

    # Annotating each GRanges object in the list with the corresponding TF ID
  motif_IDs <- motif_id_index$TF_ID
    for (i in seq_along(matches_grl)) {
        overlaps <- GenomicRanges::findOverlaps(matches_grl[[i]], merged_regions)
        overlap_indices <- subjectHits(overlaps)
        overlap_indices_query <- queryHits(overlaps)

        if (length(overlap_indices) == length(matches_grl[[i]])) {
            mcols(matches_grl[[i]]) <- mcols(merged_regions)[overlap_indices, ]
        } else {
            # Subset matches_grl[[i]] to include only overlapping ranges
            subset_matches <- matches_grl[[i]][overlap_indices_query]

            # Transfer metadata columns to the subset
            mcols(subset_matches) <- mcols(merged_regions)[overlap_indices, ]

            # Update matches_grl[[i]] with the subset
            matches_grl[[i]] <- subset_matches
        }

        matches_grl[[i]]$TF_ID <- motif_IDs[i]
    }

    return(matches_grl)
}


In [115]:
library(plyranges)
prepare_ranges_for_analysis <- function(granges_list_object, class_acrs) {
    
    
    total_bp_by_class <- granges_list_object %>% 
        group_by(conserved_regions) %>%
        summarize(total_width = sum(width))
    
    tf_counts <- as.data.frame(granges_list_object) %>% 
        rename_with(~ gsub("matches_grl.", "", .x)) %>% 
        ## Nasty, had to convert to DF before summarizing since it was a plyrange object
        left_join(., as.data.frame(total_bp_by_class), by = "conserved_regions") %>%
        mutate(acr_class_type = class_acrs)
    
    return(tf_counts)

}

In [12]:
set.seed(1234)

In [105]:
acc_acrs_file <- "/scratch/jpm73279/comparative_single_cell/07.call.ACRs/10.C4_gene_relationship_only_single_gene/all_acr_intersections/sb.extended_c4_genes.acr_intersection.all.bed"  # Replace with your file path
conserved_acr_file <- "/home/jpm73279/genome_downloads/C4_markers/conservation/Sb.all.ACR_conservation.intersection.bed"
species_abbr <- "Sb"                   # Replace with your species abbreviation
broadly_acc <- c("broadly_accessible")  # Replace with your cell type filter
ms_acc <- c("mesophyll")  # Replace with your cell type filter
bs_acc <- c("bundle_sheath")  # Replace with your cell type filter
genome <- BSgenome.Sb 

broad_acc <- process_acr_data(acc_acrs_file, 
                                   species_abbr, 
                                   broadly_acc, 
                                   markers,
                                    genome)

broad_acrs_conserved_regions_granges <- process_and_convert_conservation_data(conserved_acr_file,
                                     species_abbr,
                                      broad_acc$acr_id,
                                      genome)

broad_acc_merged_regions <- process_and_merge_granges(broad_acrs_conserved_regions_granges, broad_acc)

broad_acc_motif_locations <- match_and_annotate_motifs(pw_matrix_list_converted, 
                                             broad_acc_merged_regions, 
                                             genome,
                                             .0005)


Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 5 rows [10, 11, 12, 13,
14].”


In [107]:

mesophyll_acc <- process_acr_data(acc_acrs_file, 
                                   species_abbr, 
                                   ms_acc, 
                                   markers,
                                 genome)

mesophyll_acrs_conserved_regions_granges <- process_and_convert_conservation_data(conserved_acr_file,
                                     species_abbr,
                                      mesophyll_acc$acr_id,
                                      genome)

mesophyll_acc_merged_regions <- process_and_merge_granges(mesophyll_acrs_conserved_regions_granges, mesophyll_acc)

mesophyll_acc_motif_locations <- match_and_annotate_motifs(pw_matrix_list_converted, 
                                             mesophyll_acc_merged_regions, 
                                             genome,
                                             .0005)         


Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 5 rows [10, 11, 12, 13,
14].”


In [109]:
bundle_sheath_acc <- process_acr_data(acc_acrs_file, 
                                   species_abbr, 
                                   bs_acc, 
                                   markers,
                                     genome)

bundle_sheath_acrs_conserved_regions_granges <- process_and_convert_conservation_data(conserved_acr_file,
                                     species_abbr,
                                      bundle_sheath_acc$acr_id,
                                      genome)

bundle_sheath_acc_merged_regions <- process_and_merge_granges(bundle_sheath_acrs_conserved_regions_granges, bundle_sheath_acc)

bundle_sheath_acc_motif_locations <- match_and_annotate_motifs(pw_matrix_list_converted, 
                                             bundle_sheath_acc_merged_regions, 
                                             genome,
                                             .0005)



Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 5 rows [10, 11, 12, 13,
14].”


In [116]:

broad_acc_final <- prepare_ranges_for_analysis(broad_acc_motif_locations, "broadly_accessible")
meso_acc_final <- prepare_ranges_for_analysis(mesophyll_acc_motif_locations, "mesophyll")
bs_acc_final <- prepare_ranges_for_analysis(bundle_sheath_acc_motif_locations, "bundle_sheath")


combined_vals <- bind_rows(broad_acc_final,meso_acc_final,bs_acc_final)


ERROR: Error in UseMethod("group_by"): no applicable method for 'group_by' applied to an object of class "c('CompressedGRangesList', 'GRangesList', 'CompressedGenomicRangesList', 'GenomicRangesList', 'CompressedRangesList', 'GenomicRanges_OR_GRangesList', 'RangesList', 'GenomicRanges_OR_GenomicRangesList', 'CompressedList', 'List', 'Vector', 'list_OR_List', 'Annotated', 'vector_OR_Vector')"


In [114]:
write_delim(combined_vals, "/home/jpm73279/genome_downloads/C4_markers/count_motifs/Sb.counted_motifs.tsv",
           delim = "\t")